In [53]:
class Node:
    ''' Represents a Node in AST'''
    
    def __init__(self):
        Node.indentStep = 4      # num spaces for each indentation level
        self.depth = 0           # level of the node in tree
        self.children = []       # Child nodes for this node
        
    def addChild(self, node):
        ''' Adds the node to childern list and sets the appropriate depth of child 
        node. Returns the curr node for chaining'''
        assert isinstance (node, Node)
        self.children.append(node)
        node.depth = self.depth + 1 
        return self
        
    def _indentSpace(self):
        ''' Returns the a string of spaces for indentation'''
        spaces = ' ' * (self.depth * Node.indentStep)
        return spaces
        
    def emitEntry(self):
        ''' Returns the gen code for entry to this block as string '''
        return self._indentSpace() + '{' + '\n'
    
    def emitExit(self):
        ''' Returns the gen code for exit from this block as string '''
        return self._indentSpace() + '}' + '\n'
    
    def emitDepthOrder(self):
        ''' Visits the AST tree with this node as root return the code
        generated by depth first order traversal'''
        code = ''
        for child in self.children:
            code += child.emitEntry()
            code += child.emitDepthOrder()
            code += child.emitExit()
        return code

In [60]:
nodes = Node()

In [61]:
nodes.addChild(Node()).addChild(Node())
nodes.children[1].addChild(Node()).addChild(Node())
nodes.children[1].children[1].addChild(Node())

In [62]:
print (nodes.emitDepthOrder())

    {
    }
    {
        {
        }
        {
            {
            }
        }
    }

